In [2]:
!pip install accelerate peft datasets bitsandbytes evaluate -q
!pip install rouge_score
!pip install bert_score

# **Import library and load module**

In [3]:
import torch
import numpy as np
import evaluate
import json
import shutil
from IPython.display import FileLink
from torch.utils.data import DataLoader
from transformers import (
    LlamaForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    default_data_collator,
)
from datasets import load_dataset, Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import os

In [4]:
from huggingface_hub import login

hf_token = os.getenv("HF_TOKEN")

login(token=hf_token)

In [5]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [6]:
model_id = "meta-llama/Llama-3.2-1B"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Thiết lập pad_token và padding side
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = LlamaForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.config.use_cache = False

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [8]:
def collate_fn(batch):
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    input_ids = [item["input_ids"] for item in batch]
    attention_masks = [item["attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "labels": labels
    }

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Li

# **Continue Pretrain**

In [8]:
dataset = load_dataset("vukhai248/vietnamese_news_16k", split="train")
dataset = dataset.select(range(2000))

README.md:   0%|          | 0.00/635 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/35.1M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/4.39M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/4.39M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12806 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1601 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1601 [00:00<?, ? examples/s]

In [9]:
def preprocess_text(text):
    return text.replace("\n", " ").strip()

In [10]:
def tokenize_fn(text, max_length=2048):
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
        
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        truncation=True,
        max_length=max_length,
        padding=False,
        return_tensors="pt"
    )
    input_ids = encoding["input_ids"].squeeze(0)
    attention_mask = encoding["attention_mask"].squeeze(0)
    labels = input_ids.clone()
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [11]:
texts = dataset["content"]
tokenized_data = [tokenize_fn(text) for text in texts]
train_loader = DataLoader(
    tokenized_data,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

In [12]:
def train(model, train_loader, device, n_epochs=3, lr=1e-5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}", leave=True)
        for batch in progress_bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            
            # Forward pass with checkpointing
            def forward_with_checkpoint(*args, **kwargs):
                return torch.utils.checkpoint.checkpoint(
                    lambda *args, **kwargs: model(*args, **kwargs),
                    *args,
                    **kwargs,
                    use_reentrant=False
                )

            outputs = forward_with_checkpoint(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{n_epochs} - Loss: {avg_loss:.4f}")
train(model, train_loader, device)

Epoch 1/3:   0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Epoch 1/3: 100%|██████████| 1000/1000 [2:35:22<00:00,  9.32s/it, loss=2.72] 


Epoch 1/3 - Loss: 2.3339


Epoch 2/3: 100%|██████████| 1000/1000 [2:37:08<00:00,  9.43s/it, loss=2.06] 


Epoch 2/3 - Loss: 2.2864


Epoch 3/3: 100%|██████████| 1000/1000 [2:36:12<00:00,  9.37s/it, loss=2.25] 

Epoch 3/3 - Loss: 2.2713


In [13]:
output_dir = "/kaggle/working/llama3-qlora-continued-pretrain"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Pretrained model saved at:", output_dir)

✅ Pretrained model saved at: /kaggle/working/llama3-qlora-continued-pretrain


In [14]:

# Đường dẫn tới thư mục chứa mô hình
model_dir = '/kaggle/working/llama3-qlora-continued-pretrain'

# Tạo file nén .zip từ thư mục
zip_name = 'llama3-qlora-continued-pretrain'  # Tên file nén (không cần đuôi .zip)
shutil.make_archive(zip_name, 'zip', model_dir)

# Tạo liên kết tải xuống cho file .zip
display(FileLink(f'{zip_name}.zip'))

/kaggle/working/llama3-qlora-continued-pretrain.zip

# **Finetune with QA dataset**

In [6]:
model_id = "/kaggle/input/llama-continued-pretrain-model/transformers/default/1"

In [10]:
with open("/kaggle/input/vietnamese-squad/train-v2.0-translated.json", "r", encoding="utf-8") as f:
    vi_squad_data = json.load(f)

In [11]:
train_data = vi_squad_data[:10000]

# Lấy 2k mẫu tiếp theo cho validation
val_data = vi_squad_data[10000:12000]

print(f"📦 Train samples: {len(train_data)}")
print(f"📦 Validation samples: {len(val_data)}")

📦 Train samples: 10000
📦 Validation samples: 2000


In [12]:
print(f"📦 Train samples: {train_data[0]}")

📦 Train samples: ['Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny\'s Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".', 'Beyonce bắt đầu nổi tiếng từ khi nào?', 'Vào cuối những năm 1990']


In [13]:
train_dataset = Dataset.from_list([{"context": item[0], "question": item[1], "answer": item[2]} for item in train_data])
val_dataset = Dataset.from_list([{"context": item[0], "question": item[1], "answer": item[2]} for item in val_data])

dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": val_dataset
})

In [14]:
print(f"1. {dataset_dict['train'][0]}")
print(f"2. {dataset_dict['train'][0]['context']}")
print(f"3. {dataset_dict['train'][0]['question']}")
print(f"4. {dataset_dict['train'][0]['answer']}")

1. {'context': 'Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny\'s Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".', 'question': 'Beyonce bắt đầu nổi tiếng từ khi nào?', 'answer': 'Vào cuối những năm 1990'}
2. Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản 

In [15]:
def preprocess_qa(example, max_length=512):
    context = example["context"]
    question = example["question"]
    answer = example["answer"]
    prompt = f"### Đây là dạng câu hỏi và trả lời dựa trên nội dung ###\n\nCâu hỏi: {question}\n\nNội dung: {context}\n\nTrả lời:"
    completion = f" {answer}"
    
    prompt_ids = tokenizer(prompt, add_special_tokens=False).input_ids
    completion_ids = tokenizer(completion, add_special_tokens=False).input_ids

    input_ids = prompt_ids + completion_ids

    labels = [-100] * len(prompt_ids) + completion_ids

    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        labels = labels[:max_length]

    attention_mask = [1] * len(input_ids)
    padding_length = max_length - len(input_ids)
    
    input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
    attention_mask = attention_mask + [0] * padding_length
    labels = labels + [-100] * padding_length
    
    return {
        "input_ids": torch.tensor(input_ids),
        "attention_mask": torch.tensor(attention_mask),
        "labels": torch.tensor(labels)
    }

tokenized_dataset = dataset_dict.map(preprocess_qa)

tokenized_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [16]:
train_loader = DataLoader(
    tokenized_dataset["train"], batch_size=4, shuffle=True, collate_fn=collate_fn
)

val_loader = DataLoader(
    tokenized_dataset["test"], batch_size=4, shuffle=False, collate_fn=collate_fn
)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
n_epochs = 2

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}", leave=True)

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1}/{n_epochs} - Train Loss: {avg_loss:.4f}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"📊 Validation Loss: {avg_val_loss:.4f}")


Epoch 1/2:   0%|          | 0/2500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Epoch 1/2: 100%|██████████| 2500/2500 [3:22:25<00:00,  4.86s/it, loss=1.42]   


✅ Epoch 1/2 - Train Loss: 1.7102
📊 Validation Loss: 1.3921


Epoch 2/2:  25%|██▍       | 621/2500 [50:22<2:32:00,  4.85s/it, loss=2.36] 

In [ ]:
output_dir = "/kaggle/working/llama3-qlora-qa-finetune"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Pretrained model saved at:", output_dir)

In [ ]:
model_dir = '/kaggle/working/llama3-qlora-qa-finetune'

# Tạo file nén .zip từ thư mục
zip_name = 'llama3-qlora-qa-finetune'  # Tên file nén (không cần đuôi .zip)
shutil.make_archive(zip_name, 'zip', model_dir)

# Tạo liên kết tải xuống cho file .zip
display(FileLink(f'{zip_name}.zip'))

# **Finetune with BBC News**

In [10]:
model.gradient_checkpointing_enable()

In [11]:
data_path = "/kaggle/input/data-llama-finetune/bbc_data_llama_finetune.json"

full_dataset = load_dataset("json", data_files=data_path, split="train")
dataset = full_dataset.train_test_split(test_size=0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
def tokenize(example, max_length=2048):
    prompt = f"### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:  ### Lệnh:\n{example['prompt']}\n\nTóm tắt:\n"
    summary = example["summary"]
    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    summary_ids = tokenizer.encode(summary, add_special_tokens=False)
    
    # Kiểm tra tổng số token
    total_length = len(prompt_ids) + len(summary_ids)
    if total_length > max_length:
        overflow = total_length - max_length
        if overflow < len(prompt_ids):
            prompt_ids = prompt_ids[:-overflow]
        else:
            prompt_ids = []
    input_ids = prompt_ids + summary_ids
    attention_mask = [1] * len(input_ids)
    labels = [-100] * len(prompt_ids) + summary_ids

    padding_length = max_length - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        labels = labels + [-100] * padding_length
    else:
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }
tokenized_dataset = dataset.map(tokenize, batched=False, fn_kwargs={"max_length": 2048})
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

collator = default_data_collator

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=2,
    shuffle=True,
    collate_fn=collator
)

val_loader = DataLoader(
    tokenized_dataset["test"],
    batch_size=2,
    shuffle=False,
    collate_fn=collator
)


Map:   0%|          | 0/1914 [00:00<?, ? examples/s]

Map:   0%|          | 0/213 [00:00<?, ? examples/s]

In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Li

In [14]:
n_epochs = 3

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    pbar = tqdm(train_loader)
    total_loss = 0

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_description(f"Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} completed — Avg loss: {avg_loss:.4f}")

Epoch 1/3


Loss: 0.1281: 100%|██████████| 957/957 [3:00:37<00:00, 11.32s/it]  


✅ Epoch 1 completed — Avg loss: 0.1809
Epoch 2/3


Loss: 0.1309: 100%|██████████| 957/957 [3:00:42<00:00, 11.33s/it]  


✅ Epoch 2 completed — Avg loss: 0.1512
Epoch 3/3


Loss: 0.1048: 100%|██████████| 957/957 [3:00:42<00:00, 11.33s/it]  

✅ Epoch 3 completed — Avg loss: 0.1442


In [15]:
output_dir = "/kaggle/working/llama3-qlora-bbc-finetuned"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Finetuned model saved at:", output_dir)

✅ Finetuned model saved at: /kaggle/working/llama3-qlora-bbc-finetuned


In [16]:
model_dir = '/kaggle/working/llama3-qlora-bbc-finetuned'

# Tạo file nén .zip từ thư mục
zip_name = 'llama3-qlora-qa-finetune'  # Tên file nén (không cần đuôi .zip)
shutil.make_archive(zip_name, 'zip', model_dir)

# Tạo liên kết tải xuống cho file .zip
display(FileLink(f'{zip_name}.zip'))

/kaggle/working/llama3-qlora-qa-finetune.zip

# **Finetune with donvanban**

In [ ]:
#model_id = "/kaggle/input/llama_finetune_v1.3/transformers/default/1/data_llama_finetune_v1.3"

In [17]:
data_path = "/kaggle/input/data-llama-finetune/vi_data_llama_finetune.json"

full_dataset = load_dataset("json", data_files=data_path, split="train")
dataset = full_dataset.train_test_split(test_size=0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
def tokenize(example, max_length=2048):
    prompt = f"### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:  ### Lệnh:\n{example['prompt']}\n\nTóm tắt:\n"
    summary = example["summary"]
    
    # Mã hóa riêng prompt và summary (không thêm special tokens)
    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    summary_ids = tokenizer.encode(summary, add_special_tokens=False)
    
    # Kiểm tra tổng số token
    total_length = len(prompt_ids) + len(summary_ids)
    if total_length > max_length:
        overflow = total_length - max_length
        # Ưu tiên giữ lại phần summary; cắt bớt prompt
        if overflow < len(prompt_ids):
            prompt_ids = prompt_ids[:-overflow]
        else:
            prompt_ids = []  # Nếu quá tràn, bỏ hết prompt
    
    # Nối prompt và summary
    input_ids = prompt_ids + summary_ids
    attention_mask = [1] * len(input_ids)
    
    # Tạo labels: phần prompt được mask bằng -100, phần summary giữ nguyên token IDs
    labels = [-100] * len(prompt_ids) + summary_ids
    
    # Padding tất cả các trường về độ dài max_length
    padding_length = max_length - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        labels = labels + [-100] * padding_length
    else:
        # Nếu quá dài, cắt bớt (nên không xảy ra nhờ truncation ở trên)
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }
tokenized_dataset = dataset.map(tokenize, batched=False, fn_kwargs={"max_length": 2048})
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

collator = default_data_collator

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=2,
    shuffle=True,
    collate_fn=collator
)

val_loader = DataLoader(
    tokenized_dataset["test"],
    batch_size=2,
    shuffle=False,
    collate_fn=collator
)


Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Li

In [20]:
n_epochs = 3

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    pbar = tqdm(train_loader)
    total_loss = 0

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_description(f"Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} completed — Avg loss: {avg_loss:.4f}")

Epoch 1/3


Loss: 0.4449: 100%|██████████| 72/72 [13:48<00:00, 11.51s/it]


✅ Epoch 1 completed — Avg loss: 0.5622
Epoch 2/3


Loss: 2.0764: 100%|██████████| 72/72 [13:47<00:00, 11.49s/it]


✅ Epoch 2 completed — Avg loss: 0.4837
Epoch 3/3


Loss: 0.1682: 100%|██████████| 72/72 [13:49<00:00, 11.52s/it]

✅ Epoch 3 completed — Avg loss: 0.4461


In [21]:
output_dir = "/kaggle/working/llama3-qlora-finetuned-all"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Finetuned model saved at:", output_dir)

✅ Finetuned model saved at: /kaggle/working/llama3-qlora-finetuned-all


In [22]:
model_dir = '/kaggle/working/llama3-qlora-finetuned-all'

# Tạo file nén .zip từ thư mục
zip_name = 'llama3-qlora-donvanban-finetune'  # Tên file nén (không cần đuôi .zip)
shutil.make_archive(zip_name, 'zip', model_dir)

# Tạo liên kết tải xuống cho file .zip
display(FileLink(f'{zip_name}.zip'))

/kaggle/working/llama3-qlora-donvanban-finetune.zip

In [23]:
def evaluate_after_training(model, val_loader, tokenizer, device, num_samples=3):
    model.eval()
    predictions = []
    references = []
    samples_to_print = []

    rouge = evaluate.load("rouge")
    bert_score = evaluate.load("bertscore")

    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"]

            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=128,
                num_beams=4,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
            )

            generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            label_texts = []
            for label in labels:
                label_ids = [token_id for token_id in label.tolist() if token_id != -100]
                label_texts.append(tokenizer.decode(label_ids, skip_special_tokens=True))

            predictions.extend(generated_texts)
            references.extend(label_texts)

            if len(samples_to_print) < num_samples:
                for src, ref, pred in zip(tokenizer.batch_decode(input_ids, skip_special_tokens=True), label_texts, generated_texts):
                    samples_to_print.append((src, ref, pred))
                    if len(samples_to_print) >= num_samples:
                        break

    # ROUGE
    rouge_results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    # BERTScore (tiếng Việt)
    bert_results = bert_score.compute(predictions=predictions, references=references, lang="vi")
    bert_f1 = np.mean(bert_results["f1"])

    # In kết quả
    print("\n📊 ROUGE Scores:")
    for key in rouge_results:
        print(f"{key}: {rouge_results[key]:.4f}")

    print(f"\n📈 BERTScore F1: {bert_f1:.4f}")

    print("\n📝 Sample Results:")
    for i, (src, ref, pred) in enumerate(samples_to_print):
        print(f"\n--- Sample {i+1} ---")
        print(f"[Prompt]    {src[:300]}...")  # rút gọn prompt nếu quá dài
        print(f"[Reference] {ref}")
        print(f"[Generated] {pred}")


In [24]:
evaluate_after_training(model, val_loader, tokenizer, device)

Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Evaluating: 100%|██████████| 9/9 [05:18<00:00, 35.34s/it]


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


📊 ROUGE Scores:
rouge1: 0.2459
rouge2: 0.2449
rougeL: 0.2449
rougeLsum: 0.2463

📈 BERTScore F1: 0.7540

📝 Sample Results:

--- Sample 1 ---
[Prompt]    ### Đây là dạng tóm tắt văn bản tin tức với độ dài tóm tắt đầu ra khoảng 150 từ:  ### Lệnh:
Bạn là một trợ lý tóm tắt văn bản. Hãy cung cấp bản tóm tắt ngắn gọn và chính xác trong 150 chữ cho bài viết sau. Bài viết: ﻿Mô hình giáo dục song ngữ chuẩn quốc tế
Việc triển khai chương trình song ngữ ở các...
[Reference] ﻿Tập đoàn EMG Education hợp tác Hội đồng Khảo thí Quốc tế tạo ra mô hình giáo dục chất lượng cao, mang tính hội nhập quốc tế. 
Các chương trình của CIE dành cho học sinh phổ thông lứa tuổi tiểu học, trung học cơ sở chủ yếu tập trung vào phát triển kỹ năng cho 3 môn học: toán, tiếng Anh và khoa học.
Các chương trình của CIE được hơn 160 nước trên thế giới đưa vào giảng dạy và lấy làm chuẩn khảo thí, phù hợp với thực tiễn và khả năng của học sinh Việt Nam.
Chương trình giáo dục tiên tiến này giúp học sinh tự tin, có trách nhiệm